In [4]:
from azureml.core import Workspace, Experiment, Datastore, Dataset

In [5]:
ws=Workspace.get(name='automl-K21',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',resource_group='automl-K21RG')

In [6]:
credit_dataset=Dataset.get_by_name(ws,'credit_risk_dataset')

In [7]:
credit_dataset.to_pandas_dataframe()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,True,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,False,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,False,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,False,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,False,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,False,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,False,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,False,26


In [8]:
from azureml.core import ScriptRunConfig, Environment,Experiment



In [17]:
%%writefile $experiment_name/hyperdrive.yml

name: hyperdrive
dependencies:
- python=3.6.2
- pip: 
    - azureml-defaults
    - pandas
    - numpy 
    - scikit-learn


Overwriting hyperdrive/hyperdrive.yml


In [12]:
from azureml.core.compute import ComputeTarget
compute_target=ComputeTarget(ws,'computeclusterk21')

In [18]:
environment=Environment.from_conda_specification(name='hyperdrive_environment',file_path=experiment_name+'/hyperdrive.yml')

In [40]:
experiment_name+'/'

'hyperdrive/'

In [80]:
script_run_config=ScriptRunConfig(source_directory=experiment_name+'/',script='train.py',compute_target=compute_target,environment=environment,arguments=['--input_data',credit_dataset.as_named_input('raw_data')])

In [75]:
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, BayesianParameterSampling,choice, uniform

In [83]:
parameter_space={
    '--max_depth':choice(1,10),
    '--n_estimators':choice(10,100)
}

In [84]:
hyperconfig=HyperDriveConfig(run_config=script_run_config,hyperparameter_sampling=BayesianParameterSampling(parameter_space),primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,primary_metric_name='Accuracy',max_total_runs=10)

In [85]:
hyperdrive_exp=Experiment(ws,'hyperdrive')
hyperdrive_run=hyperdrive_exp.submit(hyperconfig)
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_ebbbe92e-c5ce-4450-b548-c064af43eff3
Web View: https://ml.azure.com/runs/HD_ebbbe92e-c5ce-4450-b548-c064af43eff3?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/automl-K21RG/workspaces/automl-K21&tid=6ad91895-de06-485e-bc51-fce126cc8530

Streaming azureml-logs/hyperdrive.txt

[2023-02-19T20:50:15.167732][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space
[2023-02-19T20:50:15.8115411Z][SCHEDULER][INFO]Scheduling job, id='HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_0' 
[2023-02-19T20:50:15.9323328Z][SCHEDULER][INFO]Scheduling job, id='HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_1' 
[2023-02-19T20:50:16.0577024Z][SCHEDULER][INFO]Scheduling job, id='HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_2' 
[2023-02-19T20:50:16.118566][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-02-19T20:50:16.2046109Z][SCHEDULER][INFO]Scheduling job, id='HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_3' 
[2023-02-19T20

{'runId': 'HD_ebbbe92e-c5ce-4450-b548-c064af43eff3',
 'target': 'computeclusterk21',
 'status': 'Completed',
 'startTimeUtc': '2023-02-19T20:50:14.716757Z',
 'endTimeUtc': '2023-02-19T20:52:15.646406Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '480667cb-8aca-451c-bfd3-8de7c945392f',
  'user_agent': 'python/3.7.0 (Windows-10-10.0.22621-SP0) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.43.0',
  'space_size': '4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',
   'amlClientModule': '[Scrubbed]',
   'amlClientFunction': '[Scrubbed]',
   'tenantId': '6ad91895-de06-485e-bc51-fce126cc8530',
   'amlClientRequestId': '75dcf75f-dba8-4ebc-874e-a40d3a327ebd',
   'amlCl

In [92]:
hyperdrive_run.get_metrics()

{'HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_2': {'accuracy': 0.7939235844713826},
 'HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_1': {'accuracy': 0.9257326990946755},
 'HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_3': {'accuracy': 0.9271137026239067},
 'HD_ebbbe92e-c5ce-4450-b548-c064af43eff3_0': {'accuracy': 0.79729937087617}}

In [9]:
import os 
experiment_name='hyperdrive'
os.makedirs(experiment_name,exist_ok=True)


In [78]:
%%writefile $experiment_name/train.py

from azureml.core import Run
from argparse import ArgumentParser

run=Run.get_context()

AP=ArgumentParser()

AP.add_argument('--max_depth',type=int)
AP.add_argument('--n_estimators',type=int)
AP.add_argument('--input_data',type=str)
args=AP.parse_args()

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
LE=LabelEncoder()

credit_dataset_df=run.input_datasets['raw_data'].to_pandas_dataframe()
credit_dataset_df['person_emp_length'].fillna(credit_dataset_df['person_emp_length'].median(),inplace=True)
credit_dataset_df['loan_int_rate'].fillna(credit_dataset_df['loan_int_rate'].median(),inplace=True)
x=credit_dataset_df.drop('loan_status',axis=1)
y=credit_dataset_df['loan_status']
x['person_home_ownership']=LE.fit_transform(x['person_home_ownership'])
x['loan_intent']=LE.fit_transform(x['loan_intent'])
x['loan_grade']=LE.fit_transform(x['loan_grade'])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
#xgb=XGBClassifier()
rfc=RandomForestClassifier(max_depth=args.max_depth,n_estimators=args.n_estimators)
fitted_model=rfc.fit(x_train,y_train)
y_pred=fitted_model.predict(x_test)
run.log('accuracy',accuracy_score(y_pred,y_test))
confusion_matrix=confusion_matrix(y_test,y_pred)
cmtx=   {
       "schema_type": "confusion_matrix",
       "schema_version": "1.0.0",
       "data": {
           "class_labels": ["0", "1", "2", "3"],
           "matrix": confusion_matrix.tolist()
           
       }
   }
run.complete()



Overwriting hyperdrive/train.py


In [45]:
from argparse import ArgumentParser
AP=ArgumentParser()

AP.add_argument('--max_depth',type=int)
AP.add_argument('--n_estimators',type=int)
AP.parse_args()


usage: ipykernel_launcher.py [-h] [--max_depth MAX_DEPTH]
                             [--n_estimators N_ESTIMATORS]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9014 --control=9012 --hb=9011 --Session.signature_scheme="hmac-sha256" --Session.key=b"5ed13d56-815d-400a-a8dd-ed6fbbce7a66" --shell=9013 --transport="tcp" --iopub=9015 --f=c:\Users\satya\AppData\Roaming\jupyter\runtime\kernel-v2-240488S2HQP7MN9D.json


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.
